In [1]:
from __future__ import print_function

import argparse
import pickle

import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, datasets

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                                ])
trainset_labeled_import = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
trainset_unlabeled_import = pickle.load(open("../data/kaggle/train_unlabeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))
train_labeled_loader = torch.utils.data.DataLoader(trainset_labeled_import, batch_size=32, shuffle=True)
train_unlabeled_loader = torch.utils.data.DataLoader(trainset_unlabeled_import, batch_size=256,
                                                     shuffle=True)

train_unlabeled_loader.dataset.train_labels = [-1 for i in range(len(train_unlabeled_loader.dataset.train_data))]
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=32, shuffle=True)

Files already downloaded
3000
750


In [171]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        print(x.size())
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)
model = Net()
a = torch.randn(32, 1, 28, 28)
model.forward(Variable(a))

torch.Size([32, 20, 4, 4])


Variable containing:
-2.3540 -2.3185 -2.3540 -2.1871 -2.3540 -2.3540 -2.3540 -2.3097 -2.3540 -2.1192
-2.3662 -2.3662 -2.3496 -2.1158 -2.3662 -2.3662 -2.3662 -2.1854 -2.3662 -2.2202
-2.3395 -2.3395 -2.3395 -2.1867 -2.3395 -2.2995 -2.3395 -2.2750 -2.3395 -2.2413
-2.3884 -2.2542 -2.3884 -2.2824 -2.3884 -2.3884 -2.3884 -2.2192 -2.3884 -2.0124
-2.3977 -2.2243 -2.3977 -2.1247 -2.3977 -2.3730 -2.3977 -2.1489 -2.3611 -2.2579
-2.3635 -2.3327 -1.9843 -2.3276 -2.4016 -2.4016 -2.4016 -2.2453 -2.4016 -2.2473
-2.2541 -2.3334 -2.2121 -2.3334 -2.3334 -2.3334 -2.3334 -2.2370 -2.3334 -2.3334
-2.3570 -2.3570 -2.3570 -2.3570 -2.3570 -2.3570 -2.1721 -2.2473 -2.3570 -2.1418
-2.2479 -2.3086 -2.2341 -2.3485 -2.2541 -2.3485 -2.3485 -2.2504 -2.3485 -2.3485
-2.3753 -2.3253 -2.1638 -2.1836 -2.4261 -2.4261 -2.4261 -2.2003 -2.4261 -2.1418
-2.3386 -2.3358 -2.3232 -2.3307 -2.3386 -2.3386 -2.1604 -2.2763 -2.3386 -2.2609
-2.3500 -2.3500 -2.3069 -2.1984 -2.3239 -2.3500 -2.3360 -2.1369 -2.3500 -2.3500
-2.3385 -2.3385 -2.

In [179]:
class NetBN(nn.Module):
    def __init__(self):
        super(NetBN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=(1, 1))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=(1, 1))
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, padding=(1, 1))
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3_bn = nn.BatchNorm2d(128)
        self.conv1_drop = nn.Dropout2d(0.2)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.conv3_drop = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(32 * 128, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.conv1_drop(F.relu(F.max_pool2d(self.conv1(x), (2,2))))
        print(x.size())

        x = self.conv2_drop(F.relu(F.max_pool2d(self.conv2(x), (2,2))))
        print(x.size())

        x = self.conv3_drop(F.relu(F.max_pool2d(self.conv3(x), (2,2))))
        print(x.size())
        x = x.view(-1, 32 * 128)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)

model = NetBN()
a = torch.randn(32, 1, 28, 28)
model.forward(Variable(a))


torch.Size([32, 32, 13, 13])
torch.Size([32, 64, 5, 5])
torch.Size([32, 64, 5, 5])


Variable containing:
-2.2338 -2.3471 -2.4053 -2.3392 -2.4053 -2.4053 -2.4053 -2.0519 -2.1661 -2.3355
[torch.FloatTensor of size 1x10]

In [166]:
model = NetBN()
optimizer = optim.SGD(model.parameters(), lr=.01, momentum=.8)

In [167]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_labeled_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)

#         loss = my_criterion.forward(output, target, Variable(torch.LongTensor(epoch)), Variable(torch.LongTensor(1)))
#         my_criterion.backward(loss)
        print(data.size())
        print(output.data.size(), target.data.size())
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Labeled Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_labeled_loader.dataset),
                       100. * batch_idx / len(train_labeled_loader), loss.data[0]))

    for batch_idx, (data,_) in enumerate(train_unlabeled_loader):
        data = Variable(data)
        optimizer.zero_grad()
        output = model(data)
        target = Variable(torch.LongTensor(output.data.max(1)[1].numpy().reshape(-1)))
#         loss = my_criterion.forward(output, target, Variable(torch.LongTensor(epoch)), Variable(torch.LongTensor(2)))
#         my_criterion.backward(loss)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Unlabeled Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_unlabeled_loader.dataset),
                       100. * batch_idx / len(train_unlabeled_loader), loss.data[0]))

In [168]:
def validate(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += my_criterion.forward(output, target, Variable(epoch), Variable(torch.LongTensor(1))).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [169]:
for epoch in range(1, 2):
    train(epoch)
    validate(epoch, valid_loader)

torch.Size([32, 128, 1, 1])
torch.Size([32, 1, 28, 28])
torch.Size([1, 10]) torch.Size([32])


RuntimeError: Assertion `THIndexTensor_(size)(target, 0) == batch_size' failed.  at /Users/soumith/anaconda/conda-bld/pytorch-0.1.7_1485439972367/work/torch/lib/THNN/generic/ClassNLLCriterion.c:50

In [ ]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
class MyCriterion(torch.autograd.Function):
    def __init__(self):
        self.alpha = .0005
    
    def forward(self, input, target, epoch, isLabeled):
        loss = F.cross_entropy(input, target)

        self.save_for_backward(input, target, epoch, isLabeled, loss)
        print(self.saved_tensors)
        if (isLabeled.data > 0).all():

            return Variable(loss.data * self.alpha * epoch.data)
        return loss

    def backward(self, grad_output):
       
        input, target, epoch, isLabeled, loss, = self.saved_tensors
        grad_input = loss.backward() 
        return grad_input
my_criterion = MyCriterion()
x = Variable(torch.randn(11, 10).type(torch.FloatTensor))
y = Variable(torch.range(1,6, .5).type(torch.LongTensor))

a = torch.from_numpy(np.array([0]))
b = torch.from_numpy(np.array([1]))
c = torch.from_numpy(np.array([10.0]))

print(x)
# print(torch.from_numpy(np.array([10])))
first_loss  = my_criterion.forward(x, y, Variable(c.float()),  Variable(a))
print(my_criterion.backward(first_loss))

second_loss = my_criterion.forward(x, y, Variable(c.float()),  Variable(b))
print(my_criterion.backward(second_loss))

In [15]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    
    label_predict = np.concatenate((label_predict, temp))

In [53]:
label_predict

predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

predict_label.head()

predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)